Import Requirements

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import albumentations as A

Set the Hyperparameters

In [2]:
model_name = "efficientnet_b0"
weights = None #"imagenet"
input_dims = (32, 32, 3)
num_classes = 10
epochs = 250
batch_size = 4112
shuffle_size = 16
learning_rate = 0.01
precision = "float32"
model_dir = "models"

Create the Models

In [3]:
"""
import resnet as resnet

inputs = tf.keras.layers.Input(shape=input_dims)
bb = resnet.ResNet50V2(input_shape=input_dims, num_classes=num_classes, pretrain=False)(inputs)
model = tf.keras.Model(inputs, bb)
"""

'\nimport resnet as resnet\n\ninputs = tf.keras.layers.Input(shape=input_dims)\nbb = resnet.ResNet50V2(input_shape=input_dims, num_classes=num_classes, pretrain=False)(inputs)\nmodel = tf.keras.Model(inputs, bb)\n'

In [4]:
models = {
    'efficientnet_b0': tf.keras.applications.EfficientNetB0,
    'efficientnet_b1': tf.keras.applications.EfficientNetB1,
    'efficientnet_b2': tf.keras.applications.EfficientNetB2,
    'efficientnet_b3': tf.keras.applications.EfficientNetB3,
    'efficientnet_b4': tf.keras.applications.EfficientNetB4,
    'efficientnet_b5': tf.keras.applications.EfficientNetB5,
    'efficientnet_b6': tf.keras.applications.EfficientNetB6,
    'efficientnet_b7': tf.keras.applications.EfficientNetB7
}
inputs = tf.keras.Input(shape=input_dims)
backbone = models[model_name](include_top=True,
                        weights=weights,
                        input_shape=input_dims,
                        classes=num_classes)(inputs)
backbone.trainable = True
# backbone = tf.keras.layers.Dense(num_classes, activation="softmax")(backbone)
model = tf.keras.Model(inputs=inputs, outputs=backbone, name=model_name)

In [5]:
model.summary()

Model: "efficientnet_b0"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 efficientnetb0 (Functional)  (None, 10)               4062381   
                                                                 
Total params: 4,062,381
Trainable params: 4,020,358
Non-trainable params: 42,023
_________________________________________________________________


Import test training data (MNIST)

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


Import Real Training Data

In [ ]:
x_train = []
y_train = []



Preprocessing

In [7]:
def augment_preprocess(image):
    image = np.array(image)
    transform = A.Compose([
        A.Flip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Resize(height=input_dims[0], width=input_dims[1]),
    ])
    transformed_image = np.array(transform(image=image)["image"])
    return transformed_image

def one_hot(indicies):
    return tf.one_hot(indicies, num_classes)[0]

In [8]:
x_training_np = np.array([augment_preprocess(image) for image in x_train])
y_training_np = np.array([one_hot(label) for label in y_train])

In [9]:
print("Image shapes" + str(x_training_np.shape))
print("Label shape" + str(y_training_np.shape))

Image shapes(50000, 32, 32, 3)
Label shape(50000, 10)


Datagenerator

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(rotation_range=5,  # rotation
                                   width_shift_range=0.2,  # horizontal shift
                                   zoom_range=0.2,  # zoom
                                   horizontal_flip=True,  # horizontal flip
                                   brightness_range=[0.2,0.8])  # brightness

Training Settings

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

Training

In [18]:
model.compile(optimizer=optimizer, loss=loss_func, metrics=["accuracy", "categorical_crossentropy"])
model.fit(train_datagen.flow(x_training_np, y_training_np, batch_size=batch_size), epochs=epochs, shuffle=True, validation_split=0.2)

Epoch 1/250
13/13 [==============================] - 25s 1s/step - loss: 2.1308 - accuracy: 0.2518 - categorical_crossentropy: 2.1308
Epoch 2/250
13/13 [==============================] - 18s 1s/step - loss: 1.9888 - accuracy: 0.2893 - categorical_crossentropy: 1.9888
Epoch 3/250
13/13 [==============================] - 18s 1s/step - loss: 1.8321 - accuracy: 0.3469 - categorical_crossentropy: 1.8321
Epoch 4/250
13/13 [==============================] - 18s 1s/step - loss: 1.6656 - accuracy: 0.3909 - categorical_crossentropy: 1.6656
Epoch 5/250
13/13 [==============================] - 18s 1s/step - loss: 1.6798 - accuracy: 0.4159 - categorical_crossentropy: 1.6798
Epoch 6/250
13/13 [==============================] - 18s 1s/step - loss: 1.5817 - accuracy: 0.4380 - categorical_crossentropy: 1.5817
Epoch 7/250
13/13 [==============================] - 18s 1s/step - loss: 1.5535 - accuracy: 0.4494 - categorical_crossentropy: 1.5535
Epoch 8/250
13/13 [==============================] - 18s 1s/st

Testing

In [ ]:
acc = tf.keras.metrics.CategoricalAccuracy()

x_test_ds = tf.data.Dataset.from_tensor_slices(x_test)
y_test_ds = tf.data.Dataset.from_tensor_slices(y_test)
test_dataset = tf.data.Dataset.zip((x_test_ds, y_test_ds)).batch(batch_size)

for x, y in test_dataset:
    out = model(x, training=False)
    acc.update_state(y_true=y[0], y_pred=out)
print(acc.result())

tf.Tensor(0.083, shape=(), dtype=float32)
